In [11]:
import time
import pyautogui
from wrt import wrt
import dotenv
import os
import pandas as pd
from datetime import date, timedelta
import win32com.client as win32
import openpyxl

In [9]:
downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")

# List all files in the Downloads folder
files = [os.path.join(downloads_folder, f) for f in os.listdir(downloads_folder)
         if os.path.isfile(os.path.join(downloads_folder, f))]

# Get the most recently created file
latest_file = max(files, key=os.path.getctime)

# print("The latest file is:", latest_file)

# Check if the latest file was modified in the last 30 seconds
# if (time.time() - os.path.getctime(latest_file)) > 30:
#     raise ValueError(f"The latest file was not modified in the last 30 seconds: {latest_file}")

# Check if the latest file is an Excel file with the .xlsx extension
if not (latest_file.lower().endswith('.xlsx')):
    raise ValueError(f"The latest file is not an Excel file: {latest_file}")

# Read the Excel file using pandas
df = pd.read_excel(latest_file, engine='openpyxl')

print("Successfully read Excel file:", latest_file)
df

Successfully read Excel file: C:\Users\yaman.bicer\Downloads\5559c844-6914-4a94-8ace-88dd84b7a7e2.xlsx


,Group Name,Organization Name,Effective Date,KGUP 4PM (MWh),KGUP Latest (MWh),Generation (MWh),Avg MCP,GIPAF,GIPSF,idm_purchase_volume,...,Final P&L,Imblance Cost wo/Synergy,Imbalance Cost wo/DSG Synergy,Final Imbalance Cost (TL),Financial Gain (TL),KUPST,Contract Revenue,GIP Net Pozisyon PNL,IDM Forecast Change Cost (TL),Avg SMP
0,ECOGREEN,ECOFER,2025-03-12 00:00:00,2.70,2.70,0.00,2304.99,0.000000,0.000000,0.0,...,-1914.869974,-2116.746000,-1914.869974,-1914.869974,0,-230.773050,0.000000,0.000000,0.000000,2999
1,GENPA,KÜTLE,2025-03-12 00:00:00,20.92,20.92,19.38,2304.99,0.000000,2339.900000,0.0,...,-1582.037698,-4405.967600,-2886.607059,-2886.607059,0,-33.999663,1161.438361,143.131000,0.000000,2999
2,RT,İNEBOLU,2025-03-12 00:00:00,172.76,86.38,57.20,2304.99,2339.900000,0.000000,22.0,...,-1411.421337,-5644.656000,-643.401337,-643.401337,0,0.000000,0.000000,-768.020000,0.000000,2999
3,ERTÜRK GRUP,TEPERES,2025-03-12 00:00:00,3.49,3.49,3.04,2304.99,0.000000,2339.900000,0.0,...,-1403.951289,-1771.794800,-1466.789289,-1466.789289,0,-102.484827,0.000000,62.838000,0.000000,2999
4,RT,SAĞANAK,2025-03-12 00:00:00,71.96,35.98,28.46,2304.99,2339.900000,0.000000,15.7,...,-917.403007,-564.261552,-369.316007,-369.316007,0,-348.435816,0.000000,-548.087000,0.000000,2999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,AKFEN,KORDA,2025-03-12 23:00:00,33.53,33.53,45.00,2174.99,0.000000,2383.300000,0.0,...,3491.372391,-456.747900,-186.510009,-186.510009,0,0.000000,2740.487400,937.395000,0.000000,2850
1868,KIVANÇ GRUP,EKİM,2025-03-12 23:00:00,13.10,13.10,42.57,2174.99,0.000000,2383.300000,0.0,...,4863.945757,-983.312979,-774.513985,-774.513985,0,-798.860700,2638.795743,2999.664000,0.000000,2850
1869,GENPA,KÜTLE,2025-03-12 23:00:00,22.58,22.58,38.19,2174.99,0.000000,2383.300000,0.0,...,5217.006315,-58.072233,-4.785255,-4.785255,0,0.000000,2159.634571,3062.157000,0.000000,2850
1870,YILDIZLAR GRUP,YELEN,2025-03-12 23:00:00,159.90,159.90,166.05,2174.99,0.000000,2383.300000,0.0,...,6987.827447,-388.235715,-96.397798,-96.397798,0,0.000000,7042.563245,41.662000,0.000000,2850


In [10]:
# Load the workbook and select the first sheet.
wb = openpyxl.load_workbook("TürkiyeDSGDailyReportTemplate.xlsx")
ws = wb.worksheets[0] # first sheet

# Update the first 22 columns.
# Here, we assume df is a DataFrame with the same number of rows as needed.
# This loop updates each cell with the corresponding value from df.
for i, row in enumerate(df.values, start=1):  # Excel rows start at 1
    for j, value in enumerate(row[:22], start=1):  # Only update columns 1 to 22
        ws.cell(row=i, column=j, value=value)

# Save the workbook. This will update the first sheet while leaving the second sheet unchanged.
wb.save(f"reports-excel/TürkiyeDSGDailyReport_{date.today() - timedelta(1)}.xlsx")
# save the second sheet of the workbook as pdf into f"reports-pdf/TürkiyeDSGDailyReport_{date.today() - timedelta(1)}.pdf"

In [13]:
excel_file = os.path.abspath(f"reports-excel/TürkiyeDSGDailyReport_{date.today() - timedelta(1)}.xlsx")
pdf_path = os.path.abspath(f"reports-pdf/TürkiyeDSGDailyReport_{date.today() - timedelta(1)}.pdf")

# Initialize Excel application
excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = False

# Open the workbook
wb = excel.Workbooks.Open(excel_file)

# Excel's Worksheets collection is 1-indexed, so the second sheet is item 2.
second_sheet = wb.Worksheets(2)

# Export the second sheet as PDF (0 represents PDF format)
second_sheet.ExportAsFixedFormat(0, pdf_path)

# Close the workbook without saving changes
wb.Close(False)

# Quit Excel
excel.Quit()